In [1]:
from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, UpSampling2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.callbacks import Callback
import random
import glob
import wandb
from wandb.keras import WandbCallback

C:\Users\hohsiny\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import subprocess
import os
from PIL import Image
import numpy as np
import cv2

run = wandb.init()
config = run.config

config.num_epochs = 1
config.batch_size = 4
config.img_dir = "images"
config.height = 256
config.width = 256

val_dir = 'test'
train_dir = 'train'

wandb: ERROR: Headless mode isn't supported on Windows. If you want to use W&B, please use "wandb run ..."; running normally.


In [20]:
# automatically get the data if it doesn't exist
if not os.path.exists("train"):
    print("Downloading flower dataset...")
    subprocess.check_output("curl https://storage.googleapis.com/l2kzone/flowers.tar | tar xz", shell=True)

def my_generator(batch_size, img_dir):
    """A generator that returns black and white images and color images"""
    image_filenames = glob.glob(img_dir + "\*")
    counter = 0
    while True:
        bw_images = np.zeros((batch_size, config.width, config.height))
        color_images = np.zeros((batch_size, config.width, config.height, 3))
        random.shuffle(image_filenames) 
        if ((counter+1)*batch_size>=len(image_filenames)):
              counter = 0
        for i in range(batch_size):
              img = Image.open(image_filenames[counter + i]).resize((config.width, config.height))
              #color_images[i] = np.array(img)
              #bw_images[i] = np.array(img.convert('L'))
              color_images[i] = np.array(img)/255  
              bw_images[i] = np.array(img.convert('L'))/255
        yield (bw_images, color_images)
        counter += batch_size 

In [21]:
(test_bw_images, test_color_images) = next(my_generator(145, train_dir))

In [22]:
test_bw_images.shape

(145, 256, 256)

In [23]:
test_color_images.shape

(145, 256, 256, 3)

In [24]:
test_bw_images[0]

array([[0.02745098, 0.02745098, 0.02745098, ..., 0.01176471, 0.01176471,
        0.01176471],
       [0.02745098, 0.02745098, 0.02745098, ..., 0.01176471, 0.01176471,
        0.01176471],
       [0.02745098, 0.02745098, 0.02745098, ..., 0.01176471, 0.01176471,
        0.01176471],
       ...,
       [0.95686275, 0.86666667, 0.81960784, ..., 0.51372549, 0.59607843,
        0.70196078],
       [0.78823529, 0.7372549 , 0.84313725, ..., 0.4627451 , 0.52156863,
        0.61568627],
       [0.61960784, 0.67058824, 0.82352941, ..., 0.39215686, 0.46666667,
        0.57647059]])

In [25]:
test_color_images[0, :, :, 0], test_color_images[0, :, :, 1], test_color_images[0, :, :, 2]

(array([[0.03137255, 0.03137255, 0.03137255, ..., 0.01176471, 0.01176471,
         0.01176471],
        [0.03137255, 0.03137255, 0.03137255, ..., 0.01176471, 0.01176471,
         0.01176471],
        [0.03137255, 0.03137255, 0.03137255, ..., 0.01176471, 0.01176471,
         0.01176471],
        ...,
        [0.99607843, 0.96078431, 0.95294118, ..., 0.67843137, 0.75686275,
         0.85882353],
        [0.91764706, 0.90588235, 1.        , ..., 0.62352941, 0.6745098 ,
         0.76078431],
        [0.81568627, 0.89411765, 1.        , ..., 0.55686275, 0.60784314,
         0.71372549]]),
 array([[0.02745098, 0.02745098, 0.02745098, ..., 0.01176471, 0.01176471,
         0.01176471],
        [0.02745098, 0.02745098, 0.02745098, ..., 0.01176471, 0.01176471,
         0.01176471],
        [0.02745098, 0.02745098, 0.02745098, ..., 0.01176471, 0.01176471,
         0.01176471],
        ...,
        [0.97254902, 0.9372549 , 0.87058824, ..., 0.45490196, 0.54901961,
         0.66666667],
        [0.8

In [11]:
model = Sequential()
model.add(Reshape((config.height,config.width,1), input_shape=(config.height,config.width)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))
#model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
#model.add(UpSampling2D((2, 2)))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 128, 64)      36928     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 256)       590080    
__________

In [ ]:
model.compile(optimizer='adam', loss='mse')

(val_bw_images, val_color_images) = next(my_generator(145, val_dir))

model.fit_generator( my_generator(config.batch_size, train_dir),
                     steps_per_epoch=20,
                     epochs=config.num_epochs, callbacks=[WandbCallback(data_type='image', predictions=16)],
                     validation_data=(val_bw_images, val_color_images))